In [78]:
import pandas as pd 


Preparo da base de dados

In [79]:
df = pd.read_csv("dataset/data.csv")
print(df.isnull().sum())
print(df.shape)

CUST_ID                               0
BALANCE                               0
BALANCE_FREQUENCY                     0
PURCHASES                             0
ONEOFF_PURCHASES                      0
INSTALLMENTS_PURCHASES                0
CASH_ADVANCE                          0
PURCHASES_FREQUENCY                   0
ONEOFF_PURCHASES_FREQUENCY            0
PURCHASES_INSTALLMENTS_FREQUENCY      0
CASH_ADVANCE_FREQUENCY                0
CASH_ADVANCE_TRX                      0
PURCHASES_TRX                         0
CREDIT_LIMIT                          1
PAYMENTS                              0
MINIMUM_PAYMENTS                    313
PRC_FULL_PAYMENT                      0
TENURE                                0
dtype: int64
(8950, 18)


Mantendo apenas colunas que selecionamos para analisar

In [80]:
colunas_selecionadas = [
    'BALANCE',                   # Samuel
    'CREDIT_LIMIT',              # João
    'PURCHASES',                  # Amanda
    'PURCHASES_FREQUENCY',       # Amanda
    'PURCHASES_TRX',             # Gabriel
    'ONEOFF_PURCHASES',           # Larissa
    'ONEOFF_PURCHASES_FREQUENCY' # Larissa
]

# Filtrando apenas essas colunas
df_filtrado = df[colunas_selecionadas].copy()

# Exibindo as primeiras linhas do novo DataFrame
print(df_filtrado.isnull().sum())
print(df_filtrado.shape)

BALANCE                       0
CREDIT_LIMIT                  1
PURCHASES                     0
PURCHASES_FREQUENCY           0
PURCHASES_TRX                 0
ONEOFF_PURCHASES              0
ONEOFF_PURCHASES_FREQUENCY    0
dtype: int64
(8950, 7)


Limpando linha com valor nulo

In [81]:
df_filtrado.dropna(inplace=True)

print(df_filtrado.isnull().sum())
print(df_filtrado.shape)

BALANCE                       0
CREDIT_LIMIT                  0
PURCHASES                     0
PURCHASES_FREQUENCY           0
PURCHASES_TRX                 0
ONEOFF_PURCHASES              0
ONEOFF_PURCHASES_FREQUENCY    0
dtype: int64
(8949, 7)


Discretização dos dados

[
  {
    "Data Type": "float64",
    "Column": "BALANCE",
    "Description": "Balance amount left in their account to make purchases",
    "Range": "[0, 19043.14]",
    "Range Explanation": ""
  },
  {
    "Data Type": "float64",
    "Column": "CREDIT_LIMIT",
    "Description": "Limit of Credit Card for user",
    "Range": "[0, 30000.00]",
    "Range Explanation": ""
  },
  {
    "Data Type": "float64",
    "Column": "ONEOFF_PURCHASES",
    "Description": "Maximum purchase amount done in one-go",
    "Range": "[0, 40761.25]",
    "Range Explanation": ""
  },
  {
    "Data Type": "float64",
    "Column": "ONEOFF_PURCHASES_FREQUENCY",
    "Description": "How frequently Purchases are happening in one-go",
    "Range": "[0, 1.00]",
    "Range Explanation": "Frequently Purchased? (1=Y, 0=N)"
  },
  {
    "Data Type": "float64",
    "Column": "PURCHASES",
    "Description": "Amount of purchases made from account",
    "Range": "[0, 49039.57]",
    "Range Explanation": ""
  },
  {
    "Data Type": "float64",
    "Column": "PURCHASES_FREQUENCY",
    "Description": "How frequently the Purchases are being made",
    "Range": "[0, 1.00]",
    "Range Explanation": "Frequently Purchased? (1=Y, 0=N)"
  },
  {
    "Data Type": "int64",
    "Column": "PURCHASES_TRX",
    "Description": "Numbe of purchase transactions made",
    "Range": "[0, 358]",
    "Range Explanation": ""
  },
]

In [ ]:
df_disc = df_filtrado.copy()

min_limit = df_filtrado["CREDIT_LIMIT"].min()
max_limit = df_filtrado["CREDIT_LIMIT"].max()

credit_limit_bins = [min_limit - 1, 1500, 5000, 15000, max_limit + 1]
credit_limit_labels = ["baixo", "médio", "alto", "muito alto"]

# BALANCE (4 faixas)
df_disc["BALANCE"] = pd.qcut(
    df_filtrado["BALANCE"], q=4, labels=["muito baixo", "baixo", "médio", "alto"]
)

# CREDIT_LIMIT (4 faixas manuais)
df_disc["CREDIT_LIMIT"] = pd.cut(
    df_filtrado["CREDIT_LIMIT"],
    bins=credit_limit_bins,
    labels=credit_limit_labels,
    include_lowest=True,
)

# PURCHASES (4 faixas) — cuidado: df_filtrado talvez não tenha essa coluna
df_disc["PURCHASES"] = pd.qcut(
    df["PURCHASES"], q=4, labels=["muito baixo", "baixo", "médio", "alto"]
)

# PURCHASES_FREQUENCY (3 faixas)
df_disc["PURCHASES_FREQUENCY"] = pd.cut(
    df_filtrado["PURCHASES_FREQUENCY"],
    bins=[-0.01, 0.33, 0.66, 1],
    labels=["baixo", "médio", "alto"],
)

# Aplica qcut sem labels primeiro para ver quantos bins são gerados
temp_bins = pd.qcut(df_filtrado["PURCHASES_TRX"], q=5, duplicates="drop")
num_bins = temp_bins.cat.categories.size

# Define os labels dinamicamente
trx_labels = ["nenhuma", "baixa", "moderada", "alta", "muito alta"][:num_bins]

# Aplica com os labels ajustados
df_disc["PURCHASES_TRX"] = pd.qcut(
    df_filtrado["PURCHASES_TRX"],
    q=5,
    labels=trx_labels,
    duplicates="drop"
)

# Gera temporariamente os bins para saber quantos foram criados de fato
temp_bins = pd.qcut(df["ONEOFF_PURCHASES"], q=4, duplicates='drop')
num_bins = temp_bins.cat.categories.size

# Ajusta os labels conforme a quantidade real de bins
labels = ["muito baixo", "baixo", "médio", "alto"][:num_bins]

# Aplica com os labels corretos
df_disc["ONEOFF_PURCHASES"] = pd.qcut(
    df["ONEOFF_PURCHASES"],
    q=4,
    labels=labels,
    duplicates='drop'
)

# ONEOFF_PURCHASES_FREQUENCY (3 faixas)
df_disc["ONEOFF_PURCHASES_FREQUENCY"] = pd.cut(
    df_filtrado["ONEOFF_PURCHASES_FREQUENCY"],
    bins=[-0.01, 0.33, 0.66, 1],
    labels=["baixo", "médio", "alto"],
)


In [83]:
# Exibir as 10 primeiras linhas da base discretizada
print(df_disc.head(10))

       BALANCE CREDIT_LIMIT    PURCHASES PURCHASES_FREQUENCY PURCHASES_TRX  \
0  muito baixo        baixo        baixo               baixo       nenhuma   
1         alto         alto  muito baixo               baixo       nenhuma   
2         alto         alto        médio                alto      moderada   
3        médio         alto         alto               baixo       nenhuma   
4        baixo        baixo  muito baixo               baixo       nenhuma   
5        médio        médio         alto                alto         baixa   
6        baixo         alto         alto                alto          alta   
7        médio        médio        médio                alto      moderada   
8        médio         alto        médio               médio         baixa   
9        baixo         alto         alto               baixo       nenhuma   

  ONEOFF_PURCHASES ONEOFF_PURCHASES_FREQUENCY  
0      muito baixo                      baixo  
1      muito baixo                      baixo

In [84]:
# Ver tipos de dados e valores únicos por coluna discretizada
for col in df_disc.columns:
    print(f"\nColuna: {col}")
    print(df_disc[col].value_counts())


Coluna: BALANCE
BALANCE
muito baixo    2238
baixo          2237
médio          2237
alto           2237
Name: count, dtype: int64

Coluna: CREDIT_LIMIT
CREDIT_LIMIT
médio         3817
alto          2746
baixo         2234
muito alto     152
Name: count, dtype: int64

Coluna: PURCHASES
PURCHASES
alto           2238
muito baixo    2237
baixo          2237
médio          2237
Name: count, dtype: int64

Coluna: PURCHASES_FREQUENCY
PURCHASES_FREQUENCY
alto     3752
baixo    3689
médio    1508
Name: count, dtype: int64

Coluna: PURCHASES_TRX
PURCHASES_TRX
nenhuma     3688
baixa       1847
alta        1740
moderada    1674
Name: count, dtype: int64

Coluna: ONEOFF_PURCHASES
ONEOFF_PURCHASES
muito baixo    4475
médio          2238
baixo          2236
Name: count, dtype: int64

Coluna: ONEOFF_PURCHASES_FREQUENCY
ONEOFF_PURCHASES_FREQUENCY
baixo    6718
médio    1123
alto     1108
Name: count, dtype: int64


In [ ]:
df_disc.to_csv("dataset/discrete_data.csv", index=False)